In [ ]:
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic.v1 import BaseModel, Field
from typing import List

In [ ]:
temperature = 0.0

class BusinessName(BaseModel):
    name: str = Field(description="The name of the business")
    rating_score: float = Field(description="The rating score of the business. 0 is the worst, 10 is the best.")

class BusinessNames(BaseModel):
    names: List[BusinessName] = Field(description="A list of business names")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=BusinessNames)

principles = """
- The name must be easy to remember.
- Use the {industry} industry and Company context to create an effective name.
- The name must be easy to pronounce.
- You must only return the name without any other text or characters.
- Avoid returning full stops, \n or any other characters.
- The maximum length of the name must be 10 characters.
"""

In [ ]:
# Chat Model Output Parser:
model = ChatOpenAI() 
template = """Generate 5 business names for a new start up company in the {industry} industry. 
You must follow the following principles: {principles}
{format_instructions}
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

In [ ]:
prompt_and_model = chat_prompt | model

result = prompt_and_model.invoke(
    {
        "principles": principles,
        "industry": "Data Science",
        "format_instructions": parser.get_format_instructions(),
    }
)
print(parser.parse(result.content))

In [ ]:
chain = chat_prompt | model | parser

result = chain.invoke(
    {
        "principles": principles,   
        "industry": "Data Science",
        "format_instructions": parser.get_format_instructions(),
    }
)
print(result)

In [ ]:
# Technically you can also write it like model() which uses the __call__ method. However the _call__ method is an older syntax.

# Or simply use model.invoke(input) whilst using .format_prompt():
chat_response_output = model.invoke(
    chat_prompt.format_prompt(
        principles=principles,
        industry="Data Science",
        format_instructions=parser.get_format_instructions(),
    ).to_messages()
)

pydantic_schema: BusinessNames = parser.parse(chat_response_output.content)
print(pydantic_schema, "\n")
print(pydantic_schema.json())